In [9]:
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Load the dataset
file_path = '/mnt/d/codes/Datasets/datasetNew.csv'
df = pd.read_csv(file_path)

# Extract resource constraints (last row)
constraint_row = df.iloc[-1, 3:].fillna(0).astype(int).values
df = df.iloc[:-1]

# Extract items and groups
items = []
groups = df['Group'].unique()
num_resources = len(constraint_row)

# Create a list of items with their resource usage and values
for _, row in df.iterrows():
    item = {
        'Group': row['Group'],
        'Item': row['Item'],
        'Value': row['Value'],
        'Resources': row[3:].values.astype(int)
    }
    items.append(item)

# Function to generate an initial solution
def initial_solution(items):
    solution = []
    groups = set(item['Group'] for item in items)
    for group in groups:
        group_items = [item for item in items if item['Group'] == group]
        solution.append(np.random.choice(group_items))
    return solution

# Function to evaluate the solution
def evaluate_solution(solution, constraints):
    group_values = {group: 0 for group in groups}
    total_resources = np.sum([item['Resources'] for item in solution], axis=0)
    valid = np.all(total_resources <= constraints)
    
    for item in solution:
        group_values[item['Group']] += item['Value']
    
    if not valid:
        return -1  # Penalty for invalid solution
    
    return max(group_values.values())  # Return the best value from the groups

# Function to generate a neighboring solution
def neighbor_solution(solution, items):
    new_solution = solution.copy()
    groups = list(set(item['Group'] for item in solution))
    group = np.random.choice(groups)
    group_items = [item for item in items if item['Group'] == group]
    
    if len(group_items) > 1:
        new_item = np.random.choice(group_items)
        idx = next(i for i, item in enumerate(new_solution) if item['Group'] == group)
        new_solution[idx] = new_item

    return new_solution

# Simulated Annealing algorithm implementation
def simulated_annealing(items, constraints, initial_temp=1000, cooling_rate=0.003, max_iterations=1000):
    current_solution = initial_solution(items)
    best_solution = current_solution
    current_value = evaluate_solution(current_solution, constraints)
    best_value = current_value
    temp = initial_temp
    
    for _ in range(max_iterations):
        new_solution = neighbor_solution(current_solution, items)
        new_value = evaluate_solution(new_solution, constraints)
        
        if new_value > current_value or np.random.rand() < np.exp((new_value - current_value) / temp):
            current_solution = new_solution
            current_value = new_value
            if new_value > best_value:
                best_solution = new_solution
                best_value = new_value
        
        temp *= (1 - cooling_rate)
    
    return best_solution, best_value

# Function to select parents for the genetic algorithm
def select_parents(population, scores, num_parents):
    # Ensure scores are valid for sorting
    if not all(isinstance(score, (int, float)) for score in scores):
        raise ValueError("Scores contain non-numeric values.")
    
    sorted_indices = np.argsort(scores)[-num_parents:]
    return [population[i] for i in sorted_indices]

# Function to perform crossover between two parents
def crossover(parent1, parent2):
    crossover_point = np.random.randint(len(parent1))
    return parent1[:crossover_point] + parent2[crossover_point:]

# Function to perform mutation
def mutate(solution, items, mutation_rate):
    if np.random.rand() < mutation_rate:
        group = np.random.choice([item['Group'] for item in solution])
        group_items = [item for item in items if item['Group'] == group]
        if group_items:
            idx = next(i for i, item in enumerate(solution) if item['Group'] == group)
            solution[idx] = np.random.choice(group_items)
    return solution

# Genetic Algorithm implementation
def genetic_algorithm(items, constraints, pop_size=50, num_generations=100, num_parents=10, mutation_rate=0.1):
    population = [initial_solution(items) for _ in range(pop_size)]
    
    for _ in range(num_generations):
        scores = [evaluate_solution(ind, constraints) for ind in population]
        
        # Ensure scores are valid for sorting
        if not all(isinstance(score, (int, float)) for score in scores):
            raise ValueError("Scores contain non-numeric values.")

        parents = select_parents(population, scores, num_parents)
        
        new_population = parents.copy()
        
        while len(new_population) < pop_size:
            parent1, parent2 = np.random.choice(len(parents), 2, replace=False)
            child = crossover(parents[parent1], parents[parent2])
            child = mutate(child, items, mutation_rate)
            new_population.append(child)
        
        population = new_population

    best_individual = max(population, key=lambda ind: evaluate_solution(ind, constraints))
    best_value = evaluate_solution(best_individual, constraints)
    return best_individual, best_value

# Run Simulated Annealing
sa_best_solution, sa_best_value = simulated_annealing(items, constraint_row)

# Run Genetic Algorithm
ga_best_solution, ga_best_value = genetic_algorithm(items, constraint_row)

# Output results
print("Simulated Annealing Results:")
print(f"Best value: {sa_best_value}")

print("\nSimulated Annealing Best Solution:")
for item in sa_best_solution:
    print(f"Group: {item['Group']}, Item: {item['Item']}, Value: {item['Value']}")

print("\nGenetic Algorithm Results:")
print(f"Best value: {ga_best_value}")

print("\nGenetic Algorithm Best Solution:")
for item in ga_best_solution:
    print(f"Group: {item['Group']}, Item: {item['Item']}, Value: {item['Value']}")

print("\nResource Constraints:")
print(constraint_row)


/tmp/ipykernel_24848/3735871357.py:12: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  constraint_row = df.iloc[-1, 3:].fillna(0).astype(int).values


Simulated Annealing Results:
Best value: 199.0

Simulated Annealing Best Solution:
Group: Group_100, Item: Item_15, Value: 56.0
Group: Group_90, Item: Item_2, Value: 27.0
Group: Group_71, Item: Item_20, Value: 13.0
Group: Group_7, Item: Item_15, Value: 131.0
Group: Group_94, Item: Item_7, Value: 187.0
Group: Group_86, Item: Item_28, Value: 176.0
Group: Group_10, Item: Item_3, Value: 35.0
Group: Group_31, Item: Item_3, Value: 52.0
Group: Group_34, Item: Item_7, Value: 72.0
Group: Group_97, Item: Item_26, Value: 126.0
Group: Group_6, Item: Item_28, Value: 139.0
Group: Group_5, Item: Item_21, Value: 103.0
Group: Group_12, Item: Item_28, Value: 12.0
Group: Group_61, Item: Item_7, Value: 130.0
Group: Group_29, Item: Item_11, Value: 112.0
Group: Group_1, Item: Item_13, Value: 19.0
Group: Group_56, Item: Item_8, Value: 60.0
Group: Group_41, Item: Item_14, Value: 76.0
Group: Group_15, Item: Item_20, Value: 57.0
Group: Group_20, Item: Item_24, Value: 106.0
Group: Group_58, Item: Item_22, Value: